<a href="https://colab.research.google.com/github/TapasKumarDutta1/IEEE-CIS-Fraud/blob/master/Cleaning_Preprocessingand_adding_columns_with_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import warnings
from google.colab import drive
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

Getting data

In [ ]:
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 52.3MB/s]
 84% 49.0M/58.3M [00:02<00:00, 11.7MB/s]
100% 58.3M/58.3M [00:02<00:00, 20.6MB/s]
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 108MB/s]
 94% 49.0M/52.2M [00:01<00:00, 20.8MB/s]
100% 52.2M/52.2M [00:01<00:00, 33.8MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 155MB/s]


In [ ]:
warnings.filterwarnings('ignore')

Loading drive

In [ ]:
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


Reading datasets

In [ ]:
# Load the training dataset from 'train_transaction.csv.zip'
trn = pd.read_csv('train_transaction.csv.zip')

# Extract the 'isFraud' column from the training dataset and store it separately as 'isFraud'
isFraud = trn['isFraud']

# Drop the 'isFraud' column from the training dataset
trn = trn.drop(columns=['isFraud'])

# Load the testing dataset from 'test_transaction.csv.zip'
tst = pd.read_csv('test_transaction.csv.zip')


Feature selection

In [ ]:

# Extract the column names of the first 55 columns (excluding 'TransactionID') from the train dataset
useful_cols = list(trn.iloc[:, :55])
useful_cols.remove('TransactionID')

# Keep only the useful columns in the train dataset
trn = trn[useful_cols]

# Keep only the useful columns in the test dataset
tst = tst[useful_cols]

Creating unique id

In [ ]:
# Add a new column "day" to the train dataset, representing the number of days from "TransactionDT"
trn["day"] = trn["TransactionDT"] // 86400

# Add a new column "day" to the test dataset, representing the number of days from "TransactionDT"
tst["day"] = tst["TransactionDT"] // 86400

# Create new empty columns "id" in both train and test datasets
trn["id"] = ""
tst["id"] = ""

# Calculate the difference "d_1" by subtracting the value in column "D1" from the "day"
trn["d_1"] = trn["day"] - trn["D1"]
tst["d_1"] = tst["day"] - tst["D1"]

# Concatenate selected columns to create the new "id" for train and test datasets
for col in ["ProductCD", "P_emaildomain", "addr1", "card1", "d_1"]:
    trn["id"] += trn[col].astype(str)
    tst["id"] += tst[col].astype(str)

# Store the "id" columns in separate variables for train and test datasets
id_trn = trn["id"]
id_tst = tst["id"]

seperate columns in continuous and categorical

In [ ]:
# List of categorical columns
categorical = [
    "ProductCD",
    "card1",
    "card2",
    "card3",
    "card4",
    "card5",
    "card6",
    "addr1",
    "addr2",
    "P_emaildomain",
    "R_emaildomain",
] + ["M" + str(i) for i in range(1, 10)]

# List of continuous columns (excluding categorical columns)
continuous = [i for i in useful_cols if i not in categorical]

Preprocessing

In [ ]:

class ContinuousConverter:
    """
    A class to transform continuous features using logarithmic transformation and standardization.

    Attributes:
        name (str): Name of the continuous feature.
        skew (float): Skewness of the continuous feature.
        log_transform (function): Logarithmic transformation function.

    Methods:
        transform(feature): Applies logarithmic transformation and standardization to the feature.
    """
    def __init__(self, name, log_transform, feature):
        """
        Initialize the ContinuousConverter.

        Args:
            name (str): Name of the continuous feature.
            log_transform (function): Logarithmic transformation function.
            feature (pandas.Series): The continuous feature to be transformed.
        """
        self.name = name
        self.skew = feature.skew()
        self.log_transform = log_transform

    def transform(self, feature):
        """
        Applies logarithmic transformation and standardization to the given feature.

        Args:
            feature (pandas.Series): The continuous feature to be transformed.

        Returns:
            pandas.Series: The transformed feature.
        """
        if self.skew > 1:
            feature = self.log_transform(feature)
        mean = feature.mean()
        std = feature.std()
        feature = (feature - mean) / (std + 1e-6)
        return feature

# Initialize lists to store transformed continuous features
continuous_trn = []
continuous_tst = []

# List to store the new id column names
id_cols = []

# Compute the mean and standard deviation for each continuous column grouped by 'id' in the training dataset
for col in tqdm(continuous):
    id_cols += [col + "_mean", col + "_std"]
    trn[col + "_mean"] = trn.groupby(["id"])[col].transform("mean")
    trn[col + "_std"] = trn.groupby(["id"])[col].transform("std")

    # Compute the mean and standard deviation for each continuous column grouped by 'id' in the testing dataset
    tst[col + "_mean"] = tst.groupby(["id"])[col].transform("mean")
    tst[col + "_std"] = tst.groupby(["id"])[col].transform("std")

# Concatenate the new id column names with the continuous columns list
continuous += id_cols

# Apply log transformation and standardization to the continuous columns in both train and test datasets
for col in tqdm(continuous):
    log = lambda x: np.log10(x + (1 - min(0, min(x))))
    converter = ContinuousConverter(col, log, trn[col])
    continuous_trn.append(converter.transform(trn[col]).astype(np.float32))
    continuous_tst.append(converter.transform(tst[col]).astype(np.float32))

# Convert the transformed continuous features to DataFrames
continuous_trn = pd.DataFrame(continuous_trn).T
continuous_trn.columns = continuous
continuous_tst = pd.DataFrame(continuous_tst).T
continuous_tst.columns = continuous

100%|██████████| 102/102 [00:13<00:00,  7.49it/s]


In [ ]:
# Calculate the sum of missing values for each row in the train dataset
continuous_trn["isna_sum"] = continuous_trn.isna().sum(axis=1)

# Calculate the sum of missing values for each row in the test dataset
continuous_tst["isna_sum"] = continuous_tst.isna().sum(axis=1)

# Standardize the 'isna_sum' column in the train dataset
mn = continuous_trn["isna_sum"].mean()
std = continuous_trn["isna_sum"].std()
continuous_trn["isna_sum"] = (continuous_trn["isna_sum"] - mn) / std

# Standardize the 'isna_sum' column in the test dataset
mn = continuous_tst["isna_sum"].mean()
std = continuous_tst["isna_sum"].std()
continuous_tst["isna_sum"] = (continuous_tst["isna_sum"] - mn) / std

# Initialize a list to store the column names with missing values
isna_cols = []

# Handle missing values for each continuous column
for col in continuous:
    # Check if there are any missing values in the column for the train dataset
    isna = continuous_trn[col].isna()

    # If missing values are present in the column, create a new binary column to indicate missingness
    if isna.sum() > 0:
        continuous_trn[col + "_isna"] = isna.astype(int)
        continuous_tst[col + "_isna"] = continuous_tst[col].isna().astype(int)

    # Fill missing values with the mean of the column in both train and test datasets
    continuous_trn[col] = continuous_trn[col].fillna(continuous_trn[col].mean())
    continuous_tst[col] = continuous_tst[col].fillna(continuous_tst[col].mean())

Handling categorical data

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def handle_categorical(df_trn, df_tst, categorical, n_values):
    """
    Handle categorical features by one-hot encoding and converting to binary features.

    Args:
        df_trn (pandas.DataFrame): Train dataset containing the categorical features.
        df_tst (pandas.DataFrame): Test dataset containing the categorical features.
        categorical (list): List of column names with categorical features.
        n_values (int): Number of unique values to keep for each categorical feature.

    Returns:
        pandas.DataFrame, pandas.DataFrame: The one-hot encoded train and test datasets.
    """
    all = []
    df_trn = df_trn[categorical]
    df_tst = df_tst[categorical]
    for col in categorical:
        all.append(
            list(df_trn[col].value_counts().iloc[: n_values - 1].index) + ["Other"]
        )
        df_trn[col] = df_trn[col].map(lambda x: x if x in all[-1] else "Other")
        df_tst[col] = df_tst[col].map(lambda x: x if x in all[-1] else "Other")
    ohe = OneHotEncoder(categories=all)
    ohe.fit(pd.concat([df_trn, df_tst]))
    df_trn = pd.DataFrame(ohe.transform(df_trn).toarray()).astype(np.float16)
    df_tst = pd.DataFrame(ohe.transform(df_tst).toarray()).astype(np.float16)
    return df_trn, df_tst

# Handle categorical features using one-hot encoding and keep 50 unique values for each categorical feature
df_trn, df_tst = handle_categorical(trn, tst, categorical, 50)

# Display the shapes of the one-hot encoded train and test datasets
df_trn.shape, df_tst.shape

((590540, 444), (506691, 444))

Concatenating continuous and categorical

In [ ]:
# Concatenate the one-hot encoded categorical features and the transformed continuous features for the train dataset
trn = pd.concat([df_trn, continuous_trn], axis=1)

# Concatenate the one-hot encoded categorical features and the transformed continuous features for the test dataset
tst = pd.concat([df_tst, continuous_tst], axis=1)

Columns created

In [ ]:
# Add the "isFraud" column back to the train dataset
trn["isFraud"] = isFraud

# Save the test dataset to a CSV file named "test_id.csv"
tst.to_csv("/content/gdrive/My Drive/fraud/test_id.csv")

# Save the train dataset to a CSV file named "train_id.csv"
trn.to_csv("/content/gdrive/My Drive/fraud/train_id.csv")